In [1]:
import os 
import openai 
from dotenv import load_dotenv 

load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")  
model_name = "gpt-3.5-turbo"

In [2]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.schema.output_parser import StrOutputParser 

In [7]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
) 
model = ChatOpenAI(model=model_name) 
output_parser = StrOutputParser() 

In [10]:
chain = prompt | model | output_parser 
chain.invoke({"topic": "bears"})

'Why did the bear bring toilet paper to the party? Because he heard it was a "bear" necessity!'

In [11]:
# More complex chain 
from langchain_openai import OpenAIEmbeddings 
from langchain.vectorstores import DocArrayInMemorySearch 

In [13]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever() 

In [14]:
retriever.get_relevant_documents("where did harrison go to work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [15]:
retriever.get_relevant_documents("what do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [18]:
template = """Answer the question based only on the following context: 
{context}

Question: {question}""" 
prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain.schema.runnable import RunnableMap 
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]), 
    "question": lambda x: x['question']
}) | prompt | model | output_parser 

In [20]:
chain.invoke({"question": "where di harrison work?"})

'Harrison worked at Kensho.'

In [21]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']), 
    "question": lambda x: x['question']
}) 
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

In [23]:
# Bind OpenAI Functions 
functions = [
    {
        "name": "weather_search", 
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object", 
            "properties": {
                "airport_code": {
                    "type": "string", 
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    }
]

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [25]:
runnable = prompt | model 
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c6f38852-511a-4762-adc0-f2859b5dc8f6-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

In [26]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [29]:
# Update the model by binding it with new functions
model = model.bind(functions=functions)

In [30]:
runnable = prompt | model

In [31]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-11ed63b0-f7da-450f-ba9b-36f43a65a126-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117})

In [32]:
# Fallbacks 
from langchain_openai import OpenAI 
import json

In [33]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
) 
simple_chain = simple_model | json.loads 

In [34]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line" 
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

In [35]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [36]:
chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "Nature's first green is gold,"},
 'poem3': {'title': 'A Dream Within a Dream',
  'author': 'Edgar Allan Poe',
  'firstLine': 'Take this kiss upon the brow!'}}

In [37]:
final_chain = simple_chain.with_fallbacks([chain])

In [38]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [39]:
# Interface 
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [40]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food?\n\nBecause they can't catch it!"

In [41]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship anymore!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [42]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 sit
 on
 the
 stove
?
 

Because
 he
 wanted
 to
 be
 a
 hot
 cross
 bear
!



In [43]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"